In [1]:
# create a clean env (conda or venv)
!conda create -n handsign python=3.11 -y
!conda activate handsign

!pip install opencv-python mediapipe numpy pandas scikit-learn tensorflow matplotlib

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [2]:
# read video
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# If MoviePy isn't installed, uncomment the next line:
!pip install moviepy

import moviepy.editor as mp
from IPython.display import display

# Load the video file
video_clip = mp.VideoFileClip("/content/drive/MyDrive/videoplayback-2.mp4")

# Display the video
display(video_clip)

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [11]:
# video_path,start,end,label
# /content/drive/MyDrive/videoplayback-2,00:00:17.2,00:00:19.6,HELLO
# /content/drive/MyDrive/videoplayback-2,00:00:24.0,00:00:26.3,SEE_YOU_LATER
# /content/drive/MyDrive/videoplayback-2,00:01:36.0,00:01:39.3,I OR ME
# /content/drive/MyDrive/videoplayback-2,00:01:46.0,00:01:47.3,FATHER
# /content/drive/MyDrive/videoplayback-2,00:01:55.0,00:01:57.3,MOTHER
# /content/drive/MyDrive/videoplayback-2,00:02:02.0,00:02:05.3,YES
# /content/drive/MyDrive/videoplayback-2,00:02:11.0,00:02:13.3,NO
# /content/drive/MyDrive/videoplayback-2,00:02:20.0,00:02:22.3,HELP
# /content/drive/MyDrive/videoplayback-2,00:02:33.0,00:02:35.3,PLEASE
# /content/drive/MyDrive/videoplayback-2,00:02:41.0,00:02:43.3,THANK YOU
# /content/drive/MyDrive/videoplayback-2,00:02:50.0,00:02:51.3,WANT
# /content/drive/MyDrive/videoplayback-2,00:03:00.0,00:03:01.3,WHAT
# /content/drive/MyDrive/videoplayback-2,00:03:07.0,00:03:09.3,DOG
# /content/drive/MyDrive/videoplayback-2,00:03:15.0,00:03:17.3,CAT

SyntaxError: only single target (not tuple) can be annotated (ipython-input-1244559417.py, line 2)

In [4]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/labels.csv")
print(df)


                                    video_path       start         end  \
0   /content/drive/MyDrive/videoplayback-2.mp4  00:00:17.2  00:00:19.6   
1   /content/drive/MyDrive/videoplayback-2.mp4  00:00:24.0  00:00:26.3   
2   /content/drive/MyDrive/videoplayback-2.mp4  00:01:36.0  00:01:39.3   
3   /content/drive/MyDrive/videoplayback-2.mp4  00:01:46.0  00:01:47.3   
4   /content/drive/MyDrive/videoplayback-2.mp4  00:01:55.0  00:01:57.3   
5   /content/drive/MyDrive/videoplayback-2.mp4  00:02:02.0  00:02:05.3   
6   /content/drive/MyDrive/videoplayback-2.mp4  00:02:11.0  00:02:13.3   
7   /content/drive/MyDrive/videoplayback-2.mp4  00:02:20.0  00:02:22.3   
8   /content/drive/MyDrive/videoplayback-2.mp4  00:02:33.0  00:02:35.3   
9   /content/drive/MyDrive/videoplayback-2.mp4  00:02:41.0  00:02:43.3   
10  /content/drive/MyDrive/videoplayback-2.mp4  00:02:50.0  00:02:51.3   
11  /content/drive/MyDrive/videoplayback-2.mp4  00:03:00.0  00:03:01.3   
12  /content/drive/MyDrive/videoplayba

In [5]:
# make_dataset.py
import os, math, cv2, mediapipe as mp, numpy as np, pandas as pd
from datetime import datetime

DATA_DIR = "data"                     # will create data/sequences and data/meta.csv
SEQ_LEN = 30                          # frames per example
os.makedirs(os.path.join(DATA_DIR, "sequences"), exist_ok=True)

def ts_to_sec(ts):
    # "HH:MM:SS.mmm" -> seconds (float)
    dt = datetime.strptime(ts, "%H:%M:%S.%f") if "." in ts else datetime.strptime(ts, "%H:%M:%S")
    return dt.hour*3600 + dt.minute*60 + dt.second + (dt.microsecond/1e6)

def read_segments(csv_path="/content/drive/MyDrive/labels.csv"):
    df = pd.read_csv(csv_path)
    # normalize times to seconds
    df["start_s"] = df["start"].apply(ts_to_sec)
    df["end_s"]   = df["end"].apply(ts_to_sec)
    return df

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, model_complexity=1,
                       min_detection_confidence=0.5, min_tracking_confidence=0.5)

def get_landmarks_both_hands(frame_bgr):
    # returns (126,) vector = 2 hands * (21*3) ; missing hand => zeros
    image_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    res = hands.process(image_rgb)
    # Collect up to two hands, each 63 float: (x,y,z) for 21 landmarks
    hands_vecs = []
    if res.multi_hand_landmarks:
        # sort by handedness (optional): right first then left for consistency
        order = []
        for i, handed in enumerate(res.multi_handedness):
            label = handed.classification[0].label.upper()  # "Left"/"Right"
            order.append((0 if label=="RIGHT" else 1, i))
        order.sort()
        for _, idx in order[:2]:
            lm = res.multi_hand_landmarks[idx]
            vec = []
            for p in lm.landmark:
                vec.extend([p.x, p.y, p.z])
            hands_vecs.append(np.array(vec, dtype=np.float32))
    # pad to 2 hands
    while len(hands_vecs) < 2:
        hands_vecs.append(np.zeros(63, dtype=np.float32))
    return np.concatenate(hands_vecs)  # (126,)

def sample_frames(cap, start_s, end_s, n=SEQ_LEN):
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    total = max(end_s - start_s, 1e-3)
    times = np.linspace(start_s, end_s, n, endpoint=False)
    frames = []
    for t in times:
        cap.set(cv2.CAP_PROP_POS_MSEC, t*1000)
        ok, frame = cap.read()
        if not ok:
            break
        frames.append(frame)
    return frames

def main():
    df = read_segments()
    rows = []
    for vid_path in df["video_path"].unique():
        if not os.path.isfile(vid_path):
            print(f"Missing video: {vid_path}")
    for i, row in df.iterrows():
        vid, start_s, end_s, label = row["video_path"], row["start_s"], row["end_s"], row["label"]
        cap = cv2.VideoCapture(vid)
        if not cap.isOpened():
            print(f"Could not open {vid}")
            continue
        frames = sample_frames(cap, start_s, end_s, SEQ_LEN)
        cap.release()
        if len(frames) < SEQ_LEN//2:
            print(f"Too few frames for {label} [{start_s}-{end_s}]")
            continue

        seq = []
        for f in frames:
            vec = get_landmarks_both_hands(f)   # (126,)
            seq.append(vec)
        seq = np.stack(seq, axis=0)             # (T, 126)

        # optional normalization: center by wrist (landmark 0 of right-hand slot)
        # Here we skip—MediaPipe already gives normalized x,y (0..1). z is relative.

        out_name = f"{label}_{i:05d}.npy"
        out_path = os.path.join(DATA_DIR, "sequences", out_name)
        np.save(out_path, seq)
        rows.append({"path": out_path, "label": label, "nframes": len(frames)})
        print("Saved", out_path)

    pd.DataFrame(rows).to_csv(os.path.join(DATA_DIR, "meta.csv"), index=False)
    print("Wrote meta.csv with", len(rows), "rows")

if __name__ == "__main__":
    main()


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



Saved data/sequences/HELLO_00000.npy
Saved data/sequences/SEE_YOU_LATER_00001.npy
Saved data/sequences/I OR ME_00002.npy
Saved data/sequences/FATHER_00003.npy
Saved data/sequences/MOTHER_00004.npy
Saved data/sequences/YES_00005.npy
Saved data/sequences/NO_00006.npy
Saved data/sequences/HELP_00007.npy
Saved data/sequences/PLEASE_00008.npy
Saved data/sequences/THANK YOU_00009.npy
Saved data/sequences/WANT_00010.npy
Saved data/sequences/WHAT_00011.npy
Saved data/sequences/DOG_00012.npy
Saved data/sequences/CAT_00013.npy
Wrote meta.csv with 14 rows


In [6]:
# Cell 1 — create the script file
%%writefile make_dataset.py
# (paste the entire make_dataset.py content here)
# tip: if you already have it above in the chat, paste it between these lines



Writing make_dataset.py


In [7]:
!python make_dataset.py

In [8]:
# ===== train.py (DROP-IN REPLACEMENT) =====
import os, json, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

META = "data/meta.csv"
SEQ_LEN = 30
FEATS = 126
EPOCHS = 25
BATCH_SIZE = 32
TARGET_MIN_PER_CLASS = 4   # oversample each class up to at least this many rows

# ---------- 1) LOAD META & CLEAN LABELS ----------
if not os.path.isfile(META):
    raise FileNotFoundError(f"Can't find {META}. Run make_dataset.py first.")

df = pd.read_csv(META)

# clean/sanitize labels (optional but handy)
df["label"] = (
    df["label"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", "_", regex=True)
)

print("== RAW per-class counts ==")
print(df["label"].value_counts(), "\n")

# ---------- 2) OVERSAMPLE SMALL CLASSES ----------
# This prevents stratified split from failing when classes have only 1 sample.
parts = []
rng = np.random.RandomState(42)
for lab, dfg in df.groupby("label", sort=False):
    need = max(0, TARGET_MIN_PER_CLASS - len(dfg))
    if need > 0:
        # sample with replacement to reach target min
        gain = dfg.sample(n=need, replace=True, random_state=42)
        parts.append(pd.concat([dfg, gain], ignore_index=True))
    else:
        parts.append(dfg)
df = pd.concat(parts, ignore_index=True).sample(frac=1.0, random_state=42).reset_index(drop=True)

print("== AFTER OVERSAMPLE per-class counts ==")
print(df["label"].value_counts(), "\n")

# ---------- 3) LOAD ALL SEQUENCES ----------
paths = df["path"].tolist()
labels = df["label"].tolist()

X = np.zeros((len(paths), SEQ_LEN, FEATS), dtype=np.float32)
for i, p in enumerate(paths):
    if not os.path.isfile(p):
        raise FileNotFoundError(f"Sequence file missing: {p}\nDid make_dataset.py run correctly?")
    X[i] = np.load(p)

# ---------- 4) PREPARE LABELS ----------
le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(le.classes_)
print(f"Found {num_classes} classes: {list(le.classes_)}\n")

# Save label mapping
label_map = {i: label for i, label in enumerate(le.classes_)}
with open("label_map.json", "w") as f:
    json.dump(label_map, f)

# ---------- 5) SPLIT DATA ----------
# Explicitly set test_size to number of classes to ensure at least one sample per class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_classes, train_size=len(y) - num_classes, stratify=y, random_state=42)

print(f"Train shapes: X={X_train.shape}, y={y_train.shape}")
print(f"Test shapes:  X={X_test.shape}, y={y_test.shape}\n")

# ---------- 6) BUILD MODEL ----------
model = models.Sequential([
    layers.Input(shape=(SEQ_LEN, FEATS)),
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(128),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# ---------- 7) TRAIN MODEL ----------
print("\nTraining model...")
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_split=0.2, verbose=1)

# ---------- 8) EVALUATE MODEL ----------
print("\nEvaluating model...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# ---------- 9) SAVE MODEL (optional) ----------
model.save("hand_sign_model.h5")
print("\nModel saved to hand_sign_model.h5")

== RAW per-class counts ==
label
HELLO            1
SEE_YOU_LATER    1
I_OR_ME          1
FATHER           1
MOTHER           1
YES              1
NO               1
HELP             1
PLEASE           1
THANK_YOU        1
WANT             1
WHAT             1
DOG              1
CAT              1
Name: count, dtype: int64 

== AFTER OVERSAMPLE per-class counts ==
label
HELLO            4
SEE_YOU_LATER    4
PLEASE           4
FATHER           4
MOTHER           4
DOG              4
THANK_YOU        4
NO               4
WHAT             4
CAT              4
HELP             4
I_OR_ME          4
WANT             4
YES              4
Name: count, dtype: int64 

Found 14 classes: [np.str_('CAT'), np.str_('DOG'), np.str_('FATHER'), np.str_('HELLO'), np.str_('HELP'), np.str_('I_OR_ME'), np.str_('MOTHER'), np.str_('NO'), np.str_('PLEASE'), np.str_('SEE_YOU_LATER'), np.str_('THANK_YOU'), np.str_('WANT'), np.str_('WHAT'), np.str_('YES')]

Train shapes: X=(42, 30, 126), y=(42,)
Test shapes:  X=(

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 156,878 (612.80 KB)

 Trainable params: 156,878 (612.80 KB)

 Non-trainable params: 0 (0.00 B)


Training model...
Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 466ms/step - accuracy: 0.0000e+00 - loss: 2.6490 - val_accuracy: 0.2222 - val_loss: 2.6204
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - accuracy: 0.1225 - loss: 2.6008 - val_accuracy: 0.1111 - val_loss: 2.6031
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0612 - loss: 2.5768 - val_accuracy: 0.1111 - val_loss: 2.5969
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.1531 - loss: 2.5463 - val_accuracy: 0.2222 - val_loss: 2.6059
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.2039 - loss: 2.5211 - val_accuracy: 0.1111 - val_loss: 2.6278
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.1531 - loss: 2.4878 - val_accuracy: 0.1111 - val_loss: 2.6458
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.3368 - loss: 2.4524 - val_accuracy: 0.1111 - val_loss: 2.6537
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.2449 - loss: 2.4275 - val_accura

Test Loss: 1.7986
Test Accuracy: 0.5000

Model saved to hand_sign_model.h5


In [14]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Settings — change these as needed
CSV_PATH = "/content/drive/MyDrive/labels.csv"        # <-- your CSV path
OUT_DIR  = "/content/drive/MyDrive/label_clips"       # <-- where clips go
TARGET_LABEL = "SEE_YOU_LATER FATHER"                                # <-- choose a label

# 3) Dependencies & imports
import os, subprocess, shlex, pathlib
import pandas as pd

# Ensure output folder exists
os.makedirs(OUT_DIR, exist_ok=True)

# 4) Load CSV (expects columns: video_path,start,end,label)
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip().lower() for c in df.columns]
required = {"video_path","start","end","label"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV missing columns: {missing}")

# Drop incomplete rows
df = df.dropna(subset=["video_path","start","end","label"])
df["video_path"] = df["video_path"].astype(str).str.strip()
df["label"] = df["label"].astype(str).str.strip()
df["start"] = df["start"].astype(str).str.strip()
df["end"]   = df["end"].astype(str).str.strip()

# 5) Small helper: if path has no .mp4 and file doesn’t exist, try +".mp4"
def resolve_video_path(p):
    if os.path.exists(p):
        return p
    if not p.lower().endswith(".mp4") and os.path.exists(p + ".mp4"):
        return p + ".mp4"
    return p  # return original (ffmpeg will fail if truly missing)

# 6) Clip function (re-encodes for accurate cuts)
def clip_row(row, idx=None):
    src = resolve_video_path(row["video_path"])
    if not os.path.exists(src):
        print(f"SKIP (missing source): {src}")
        return None

    label = row["label"]
    start = row["start"]   # e.g., 00:01:46.0
    end   = row["end"]     # e.g., 00:01:47.3

    # Safe filename parts
    safe_label = "".join(c if c.isalnum() or c in ("_","-") else "_" for c in label)
    base = pathlib.Path(src).stem
    idx_suffix = f"__{idx:03d}" if idx is not None else ""
    out_name = f"{safe_label}__{base}__{start.replace(':','-')}__to__{end.replace(':','-')}{idx_suffix}.mp4"
    out_path = os.path.join(OUT_DIR, out_name)

    # ffmpeg command (accurate cuts by seeking after -i; re-encodes to h264/aac)
    cmd = (
        f"ffmpeg -y -hide_banner -loglevel error "
        f"-i {shlex.quote(src)} -ss {shlex.quote(start)} -to {shlex.quote(end)} "
        f"-c:v libx264 -preset veryfast -crf 18 -c:a aac -movflags +faststart "
        f"{shlex.quote(out_path)}"
    )
    ret = subprocess.run(cmd, shell=True)
    if ret.returncode == 0:
        print("WROTE", out_path)
        return out_path
    else:
        print("FFMPEG FAILED:", out_path)
        return None

# 7) Filter rows for the chosen label and export
subset = df[df["label"].str.casefold() == TARGET_LABEL.casefold()].reset_index(drop=True)
if subset.empty:
    print(f"No rows found for label: {TARGET_LABEL}")
else:
    for i, row in subset.iterrows():
        clip_row(row, idx=i+1)

print("Done.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No rows found for label: SEE_YOU_LATER FATHER
Done.


In [16]:
# === SETTINGS ===
CSV_PATH   = "/content/drive/MyDrive/labels.csv"
OUT_DIR    = "/content/drive/MyDrive/label_clips"
LABELS     = ["MOTHER", "FATHER"]   # <- multiple labels here
MERGED_OUT = "/content/drive/MyDrive/label_clips/MOTHER__FATHER__merged.mp4"

import os, subprocess, shlex, pathlib
import pandas as pd

os.makedirs(OUT_DIR, exist_ok=True)

# --- Load & filter rows for the labels ---
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip().lower() for c in df.columns]
df = df.dropna(subset=["video_path","start","end","label"])

labels_lower = [l.casefold() for l in LABELS]
subset = df[df["label"].astype(str).str.casefold().isin(labels_lower)].copy()

# sort by start time so the merged video plays in order
def t2s(t):
    hh, mm, ss = t.split(":")
    return int(hh)*3600 + int(mm)*60 + float(ss)
subset["start_s"] = subset["start"].astype(str).map(t2s)
subset = subset.sort_values(["start_s"]).reset_index(drop=True)

def resolve_video_path(p):
    p = str(p).strip()
    if os.path.exists(p): return p
    if not p.lower().endswith(".mp4") and os.path.exists(p + ".mp4"):
        return p + ".mp4"
    return p

def safe_name(s):
    return "".join(c if c.isalnum() or c in ("_","-") else "_" for c in str(s))

def clip_row(row, idx=None):
    src = resolve_video_path(row["video_path"])
    if not os.path.exists(src):
        print("SKIP (missing source):", src); return None
    start, end, label = str(row["start"]), str(row["end"]), str(row["label"])
    base = pathlib.Path(src).stem
    idx_suffix = f"__{idx:03d}" if idx else ""
    out_name = f"{safe_name(label)}__{base}__{start.replace(':','-')}__to__{end.replace(':','-')}{idx_suffix}.mp4"
    out_path = os.path.join(OUT_DIR, out_name)

    # Export the segment (re-encodes for accurate cuts)
    cmd = (
        f"ffmpeg -y -hide_banner -loglevel error "
        f"-i {shlex.quote(src)} -ss {shlex.quote(start)} -to {shlex.quote(end)} "
        f"-c:v libx264 -preset veryfast -crf 18 -c:a aac -movflags +faststart "
        f"{shlex.quote(out_path)}"
    )
    ok = (subprocess.run(cmd, shell=True).returncode == 0)
    return out_path if ok else None

# --- ensure segments exist (creates them now) ---
clip_paths = []
for i, row in subset.iterrows():
    p = clip_row(row, idx=i+1)
    if p: clip_paths.append(p)

if not clip_paths:
    raise SystemExit("No clips were created. Check labels & paths in the CSV.")

# --- build concat list file ---
list_txt = os.path.join(OUT_DIR, "_concat_list.txt")
with open(list_txt, "w") as f:
    for p in clip_paths:
        f.write(f"file '{p}'\n")

# --- merge: fast path (no re-encode); if it fails, fallback to safe re-encode ---
cmd_fast = f"ffmpeg -y -hide_banner -loglevel error -f concat -safe 0 -i {shlex.quote(list_txt)} -c copy {shlex.quote(MERGED_OUT)}"
ret = subprocess.run(cmd_fast, shell=True).returncode

if ret != 0:
    print("Fast concat failed; trying safe re-encode…")
    cmd_safe = (f"ffmpeg -y -hide_banner -loglevel error -f concat -safe 0 "
                f"-i {shlex.quote(list_txt)} -c:v libx264 -preset veryfast -crf 18 "
                f"-c:a aac -movflags +faststart {shlex.quote(MERGED_OUT)}")
    ret = subprocess.run(cmd_safe, shell=True).returncode

print("MERGED:", MERGED_OUT if ret == 0 else "FAILED")


MERGED: /content/drive/MyDrive/label_clips/MOTHER__FATHER__merged.mp4
